In [1]:
!pip install tensorflow opencv-python matplotlib

In [2]:
!pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
astunparse                   1.6.3
certifi                      2024.6.2
charset-normalizer           3.3.2
contourpy                    1.2.1
cycler                       0.12.1
flatbuffers                  24.3.25
fonttools                    4.53.0
gast                         0.5.4
google-pasta                 0.2.0
grpcio                       1.64.1
h5py                         3.11.0
idna                         3.7
importlib_metadata           7.1.0
importlib_resources          6.4.0
keras                        3.3.3
kiwisolver                   1.4.5
libclang                     18.1.1
Markdown                     3.6
markdown-it-py               3.0.0
MarkupSafe                   2.1.5
matplotlib                   3.9.0
mdurl                        0.1.2
ml-dtypes                    0.3.2
namex                        0.0.8
numpy                        1.26.4
openc

In [3]:
import tensorflow as tf
import os

In [4]:
import cv2
import imghdr

/var/folders/q_/fg1mgg754tsb416pc5slmlt40000gn/T/ipykernel_84069/4232469594.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [5]:
data_dir = 'data' 

In [6]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [7]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [8]:
!find . -name "*.DS_Store" -type f -delete

In [51]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [52]:
dataset_dir = 'data'
train_dir = 'data/train'
val_dir = 'data/validation'
test_dir = 'data/test'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


In [57]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the paths
dataset_dir = 'data'
train_dir = 'data/train'
val_dir = 'data/validation'
test_dir = 'data/test'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

def split_data(class_dir, train_size=0.7, val_size=0.15, test_size=0.15):
    images = os.listdir(class_dir)
    if len(images) < 3:
        raise ValueError(f"Not enough samples in class {os.path.basename(class_dir)} to split. Need at least 3 samples.")
    
    train_and_val, test = train_test_split(images, test_size=test_size, random_state=42)
    train, val = train_test_split(train_and_val, test_size=val_size/(train_size + val_size), random_state=42)
    
    return train, val, test

def copy_files(file_list, source_dir, target_dir):
    for file_name in file_list:
        src = os.path.join(source_dir, file_name)
        dst = os.path.join(target_dir, file_name)
        shutil.copy(src, dst)

# Iterate over each class in the dataset
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    if not os.path.isdir(class_dir):
        continue

    try:
        train, val, test = split_data(class_dir)
    except ValueError as e:
        print(e)
        continue
    
    # Create class directories in train, val, and test folders
    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)
    
    # Copy files to their respective directories
    copy_files(train, class_dir, train_class_dir)
    copy_files(val, class_dir, val_class_dir)
    copy_files(test, class_dir, test_class_dir)


Not enough samples in class test to split. Need at least 3 samples.


IsADirectoryError: [Errno 21] Is a directory: 'data/train/happy'

# Load data

In [37]:
import numpy as np
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [38]:
data = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [39]:
train_generator = data.flow_from_directory(
        'data',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 534 images belonging to 4 classes.


In [46]:
len(train_generator)

17

In [16]:
# lable
# 0 ind= angry
# 1 ind= calm
# 2 ind= happy
# 3 ind= sad

In [22]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
        'data',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

In [23]:
train_size + val_size + test_size

17

In [24]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [25]:
train_iterator = train.as_numpy_iterator()

In [26]:
train_iterator.next()[1]

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]], dtype=float32)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [28]:
model = Sequential()

In [36]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

ValueError: Input 0 of layer "conv2d_3" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 4)

In [31]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 2e-5),
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = [tf.keras.metrics.Accuracy()])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 16)        0

In [33]:
logdir='logs'

In [34]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [35]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/saksham/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/saksham/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/Users/saksham/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/Users/saksham/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/saksham/anaconda3/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/Users/saksham/anaconda3/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/Users/saksham/anaconda3/lib/pyt